In [1]:
import torch
import torch.nn as nn

# Batch norm

In [9]:
# Criando 1 batch, 4 linhas, 2 colunas
tensor = torch.tensor([[1, 2],
                       [3, 4],
                       [5, 6],
                       [7, 8]], dtype=torch.bfloat16)
print(tensor)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]], dtype=torch.bfloat16)


In [10]:
tensor.shape

torch.Size([4, 2])

In [11]:
# Atenção você não pode ter a dim do batch só das linhas, colunas
batch_norm = nn.BatchNorm1d(num_features=2, affine=False)

In [14]:
batchnorm_tensor = batch_norm(tensor)
print(batchnorm_tensor)

tensor([[-1.3438, -1.3438],
        [-0.4473, -0.4473],
        [ 0.4473,  0.4473],
        [ 1.3438,  1.3438]], dtype=torch.bfloat16)


In [16]:
# Calculando a média e a variância do lote
mean_batch = tensor.mean(dim=0, keepdim=True)
var_batch = tensor.var(dim=0, keepdim=True, unbiased=False)
print(f'Mean Batch:\n{mean_batch}')
print(f'Var Batch:\n{var_batch}')

Mean Batch:
tensor([[4., 5.]], dtype=torch.bfloat16)
Var Batch:
tensor([[5., 5.]], dtype=torch.bfloat16)


In [19]:
# Aplicando a normalização em lote manualmente
manual_batchnorm = (tensor - mean_batch) / torch.sqrt(var_batch)
print(manual_batchnorm)

tensor([[-1.3438, -1.3438],
        [-0.4473, -0.4473],
        [ 0.4473,  0.4473],
        [ 1.3438,  1.3438]], dtype=torch.bfloat16)


In [20]:
manual_batchnorm == batchnorm_tensor

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True]])

# Layer Norm

In [23]:
# exemplo com imagem (B, C, H, W): (2, 3, 2, 2)
# 1º batch
tensor = torch.tensor([[[[1, 2],  # 1º canal 1º batch
                          [0, 0]],

                         [[0, 1],  # 2º canal 1º batch
                          [0, 0]],
                
                         [[2, 1],  # 3º canal 1º batch
                          [1, 1]]],

# 2º batch
                        [[[2, 0],  # 1º canal 2º batch
                          [1, 1]],

                         [[2, 0],  # 2º canal 2º batch
                          [2, 2]],

                         [[1, 2],  # 3º canal 2º batch
                          [2, 2]]]], dtype=torch.bfloat16)

In [25]:
tensor.shape

torch.Size([2, 3, 2, 2])

In [77]:
# como eu quero normalizar por canal eu coloco normalized_shape=(3, 2, 2) Pois eu não estou olhando para o Batch por isso eu removo o 2,
layer_norm = nn.LayerNorm(normalized_shape=(3, 2, 2), elementwise_affine=False)

In [78]:
layernorm_tensor = layer_norm(tensor)

In [79]:
# removo a dim do batch por isso não coloco o 0
mean_layer = tensor.mean(dim=(1, 2, 3), keepdim=True)
print(mean_layer)
var_layer = tensor.var(dim=(1, 2, 3), keepdim=True, unbiased=False)
print(var_layer)

tensor([[[[0.7500]]],


        [[[1.4141]]]], dtype=torch.bfloat16)
tensor([[[[0.5195]]],


        [[[0.5781]]]], dtype=torch.bfloat16)


In [80]:
# Normalizando manualmente
manual_batchnorm = (tensor - mean_layer) / torch.sqrt(var_layer + layer_norm.eps)

In [81]:
print(f'Manualmente:\n{manual_batchnorm}')
print(f'\nUsando LayerNorm:\n{layernorm_tensor}')
print('\nElas estão muito proximas mas não são identicas, porque?')

Manualmente:
tensor([[[[ 0.3457,  1.7266],
          [-1.0391, -1.0391]],

         [[-1.0391,  0.3457],
          [-1.0391, -1.0391]],

         [[ 1.7266,  0.3457],
          [ 0.3457,  0.3457]]],


        [[[ 0.7695, -1.8594],
          [-0.5430, -0.5430]],

         [[ 0.7695, -1.8594],
          [ 0.7695,  0.7695]],

         [[-0.5430,  0.7695],
          [ 0.7695,  0.7695]]]], dtype=torch.bfloat16)

Usando LayerNorm:
tensor([[[[ 0.3457,  1.7344],
          [-1.0391, -1.0391]],

         [[-1.0391,  0.3457],
          [-1.0391, -1.0391]],

         [[ 1.7344,  0.3457],
          [ 0.3457,  0.3457]]],


        [[[ 0.7695, -1.8672],
          [-0.5469, -0.5469]],

         [[ 0.7695, -1.8672],
          [ 0.7695,  0.7695]],

         [[-0.5469,  0.7695],
          [ 0.7695,  0.7695]]]], dtype=torch.bfloat16)

Elas estão muito proximas mas não são identicas, porque?


In [65]:
# Exemplo com texto ()
# 2: sequencias, 3: tokens, 5: embeddings
X = torch.randint(0, 10, (2, 3, 5), dtype=torch.bfloat16)
normalized_shape = (5,)  # normalização feita na dimensão do embedding
layer_norm = nn.LayerNorm(normalized_shape=normalized_shape, elementwise_affine=False)

In [66]:
X

tensor([[[2., 4., 4., 0., 2.],
         [6., 2., 9., 9., 4.],
         [1., 5., 0., 5., 1.]],

        [[3., 6., 9., 7., 1.],
         [6., 5., 9., 1., 5.],
         [6., 6., 8., 2., 6.]]], dtype=torch.bfloat16)

In [67]:
layer_norm(X)

tensor([[[-0.2676,  1.0703,  1.0703, -1.6016, -0.2676],
         [ 0.0000, -1.4531,  1.0859,  1.0859, -0.7266],
         [-0.6484,  1.2031, -1.1172,  1.2031, -0.6484]],

        [[-0.7695,  0.2793,  1.3281,  0.6289, -1.4688],
         [ 0.3125, -0.0781,  1.4844, -1.6406, -0.0781],
         [ 0.2041,  0.2041,  1.2266, -1.8359,  0.2041]]], dtype=torch.bfloat16)

In [74]:
# provando manualmente
mean_layer = X.mean(dim=2, keepdims=True)
var_layer = X.var(dim=2, keepdims=True, unbiased=False)

In [75]:
manual_layer = (X - mean_layer) / torch.sqrt(var_layer + layer_norm.eps)

In [76]:
# novamente chega muito proximo mas não bate nas casas decimais
manual_layer

tensor([[[-0.2715,  1.0703,  1.0703, -1.6094, -0.2715],
         [ 0.0000, -1.4531,  1.0938,  1.0938, -0.7266],
         [-0.6523,  1.2031, -1.1172,  1.2031, -0.6523]],

        [[-0.7656,  0.2832,  1.3359,  0.6328, -1.4609],
         [ 0.3164, -0.0732,  1.4844, -1.6328, -0.0732],
         [ 0.2070,  0.2070,  1.2266, -1.8359,  0.2070]]], dtype=torch.bfloat16)

Em outras palavras, se você está operando em um tensor tridimensional, como no seu caso onde cada "matriz" representa uma frase e cada linha é um token com seu embedding, o normalized_shape deve ser um tuple que corresponde ao número de dimensões que você deseja normalizar.

Por exemplo, se você tem um tensor com shape (batch_size, sequence_length, embedding_size), e deseja normalizar ao longo das dimensões dos embeddings (ou seja, ao longo da dimensão do embedding_size), você deve definir normalized_shape como (embedding_size,).

Se você deseja normalizar ao longo de todas as dimensões, você pode simplesmente definir normalized_shape como o número total de dimensões no tensor.

No seu caso específico, onde cada "matriz" representa uma frase e cada linha é um token com seu embedding, o normalized_shape deve ser (num_columns,), onde num_columns é o número de colunas em cada "matriz" (ou seja, o número de dimensões dos embeddings).

# Instance Norm

In [268]:
# exemplo com imagem (B, C, H, W): (2, 3, 2, 2)
# 1º batch
tensor = torch.tensor([[[[1, 2],  # 1º canal 1º batch
                          [0, 0]],

                         [[0, 1],  # 2º canal 1º batch
                          [0, 0]],
                
                         [[2, 1],  # 3º canal 1º batch
                          [1, 1]]],

# 2º batch
                        [[[2, 0],  # 1º canal 2º batch
                          [1, 1]],

                         [[2, 0],  # 2º canal 2º batch
                          [2, 2]],

                         [[1, 2],  # 3º canal 2º batch
                          [2, 2]]]], dtype=torch.bfloat16)

In [82]:
# num_features: nº de canais nesse caso 3
instance_norm = nn.InstanceNorm2d(num_features=3, affine=True)

In [83]:
instance_norm(tensor)

tensor([[[[ 0.3008,  1.5078],
          [-0.9062, -0.9062]],

         [[-0.5781,  1.7344],
          [-0.5781, -0.5781]],

         [[ 1.7344, -0.5781],
          [-0.5781, -0.5781]]],


        [[[ 1.4141, -1.4141],
          [ 0.0000,  0.0000]],

         [[ 0.5781, -1.7344],
          [ 0.5781,  0.5781]],

         [[-1.7344,  0.5781],
          [ 0.5781,  0.5781]]]], dtype=torch.bfloat16, grad_fn=<ViewBackward0>)

# Group Norm

In [96]:
# exemplo com imagem (B, C, H, W): (2, 3, 2, 2)
# 1º batch
tensor = torch.randint(0, 4, (2, 8, 2, 2), dtype=torch.bfloat16)

In [97]:
tensor

tensor([[[[0., 3.],
          [0., 1.]],

         [[0., 0.],
          [2., 3.]],

         [[1., 2.],
          [3., 1.]],

         [[0., 1.],
          [1., 1.]],

         [[0., 3.],
          [1., 0.]],

         [[3., 2.],
          [1., 2.]],

         [[3., 3.],
          [0., 3.]],

         [[3., 3.],
          [2., 2.]]],


        [[[0., 3.],
          [3., 1.]],

         [[3., 1.],
          [2., 0.]],

         [[2., 2.],
          [2., 2.]],

         [[2., 1.],
          [1., 1.]],

         [[3., 0.],
          [0., 2.]],

         [[2., 1.],
          [3., 2.]],

         [[0., 1.],
          [2., 2.]],

         [[3., 1.],
          [3., 0.]]]], dtype=torch.bfloat16)

In [98]:
# Separando 8 canais em 2 grupos
group_norm = nn.GroupNorm(num_groups=2, num_channels=8)
group_norm(tensor)

tensor([[[[-1.1094,  1.6875],
          [-1.1094, -0.1748]],

         [[-1.1094, -1.1094],
          [ 0.7578,  1.6875]],

         [[-0.1748,  0.7578],
          [ 1.6875, -0.1748]],

         [[-1.1094, -0.1748],
          [-0.1748, -0.1748]],

         [[-1.6953,  0.9297],
          [-0.8203, -1.6953]],

         [[ 0.9297,  0.0547],
          [-0.8203,  0.0547]],

         [[ 0.9297,  0.9297],
          [-1.6953,  0.9297]],

         [[ 0.9297,  0.9297],
          [ 0.0547,  0.0547]]],


        [[[-1.7500,  1.4844],
          [ 1.4844, -0.6758]],

         [[ 1.4844, -0.6758],
          [ 0.4043, -1.7500]],

         [[ 0.4043,  0.4043],
          [ 0.4043,  0.4043]],

         [[ 0.4043, -0.6758],
          [-0.6758, -0.6758]],

         [[ 1.2891, -1.3984],
          [-1.3984,  0.3926]],

         [[ 0.3926, -0.5039],
          [ 1.2891,  0.3926]],

         [[-1.3984, -0.5039],
          [ 0.3926,  0.3926]],

         [[ 1.2891, -0.5039],
          [ 1.2891, -1.3984]]]], dtype

In [100]:
# Separando 8 canais em 8 grupos (Equivalente ao IntanceNorm)
group_norm = nn.GroupNorm(num_groups=8, num_channels=8)
group_norm(tensor)

tensor([[[[-0.8164,  1.6328],
          [-0.8164,  0.0000]],

         [[-0.9609, -0.9609],
          [ 0.5781,  1.3438]],

         [[-0.9062,  0.3008],
          [ 1.5078, -0.9062]],

         [[-1.7344,  0.5781],
          [ 0.5781,  0.5781]],

         [[-0.8164,  1.6328],
          [ 0.0000, -0.8164]],

         [[ 1.4141,  0.0000],
          [-1.4141,  0.0000]],

         [[ 0.5781,  0.5781],
          [-1.7344,  0.5781]],

         [[ 1.0000,  1.0000],
          [-1.0000, -1.0000]]],


        [[[-1.3438,  0.9609],
          [ 0.9609, -0.5781]],

         [[ 1.3438, -0.4473],
          [ 0.4473, -1.3438]],

         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 1.7344, -0.5781],
          [-0.5781, -0.5781]],

         [[ 1.3438, -0.9609],
          [-0.9609,  0.5781]],

         [[ 0.0000, -1.4141],
          [ 1.4141,  0.0000]],

         [[-1.5078, -0.3008],
          [ 0.9062,  0.9062]],

         [[ 0.9609, -0.5781],
          [ 0.9609, -1.3438]]]], dtype

In [102]:
instance_norm = nn.InstanceNorm2d(num_features=8)
instance_norm(tensor) == group_norm(tensor)

tensor([[[[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[False,  True],
          [False, False]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]]],


        [[[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True]],

         [[False,  True],
          [False, False]],

         [[ True,  True],
          [ True,  True]]]])

In [103]:
# Separando 8 canais em 1 grupo (equivalente ao LayerNorm)
group_norm = nn.GroupNorm(num_channels=8, num_groups=1)
group_norm(tensor)

tensor([[[[-1.3359,  1.2266],
          [-1.3359, -0.4805]],

         [[-1.3359, -1.3359],
          [ 0.3730,  1.2266]],

         [[-0.4805,  0.3730],
          [ 1.2266, -0.4805]],

         [[-1.3359, -0.4805],
          [-0.4805, -0.4805]],

         [[-1.3359,  1.2266],
          [-0.4805, -1.3359]],

         [[ 1.2266,  0.3730],
          [-0.4805,  0.3730]],

         [[ 1.2266,  1.2266],
          [-1.3359,  1.2266]],

         [[ 1.2266,  1.2266],
          [ 0.3730,  0.3730]]],


        [[[-1.5547,  1.3672],
          [ 1.3672, -0.5781]],

         [[ 1.3672, -0.5781],
          [ 0.3965, -1.5547]],

         [[ 0.3965,  0.3965],
          [ 0.3965,  0.3965]],

         [[ 0.3965, -0.5781],
          [-0.5781, -0.5781]],

         [[ 1.3672, -1.5547],
          [-1.5547,  0.3965]],

         [[ 0.3965, -0.5781],
          [ 1.3672,  0.3965]],

         [[-1.5547, -0.5781],
          [ 0.3965,  0.3965]],

         [[ 1.3672, -0.5781],
          [ 1.3672, -1.5547]]]], dtype

In [105]:
layer_norm = nn.LayerNorm(normalized_shape=(8, 2, 2), elementwise_affine=False)
layer_norm(tensor) == group_norm(tensor)

tensor([[[[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]]],


        [[[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]],

         [[True, True],
          [True, True]]]])

# Utilizando nn.functional F

Para utilizar o `nn.functional` você irá precisar da media e da variancia do seu tensor:
``` python
import torch.nn.functional as F
F.batch_norm()
F.layer_norm()
F.instance_norm()
F.group_norm()
```